In [10]:
datapath = "/content/drive/MyDrive/Vigilant - Realisation/WP 4 - Automated content analysis/Machine-generated text detection/"
column_order = ['text', 'label', 'length', 'source', 'language', 'domain', 'topic']

#Installs and imports

In [1]:
!pip install datasets >/dev/null 2>&1

In [2]:
!pip install fastBPE sacremoses >/dev/null 2>&1

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
import glob
import shutil

In [5]:
from datasets import load_dataset

#Dataset Clearing

In [7]:
def clear_dataset(df):
  df.replace(r'^ ', '', regex=True, inplace=True)
  df.replace(r' $', '', regex=True, inplace=True)
  df.replace(r'^\t', '', regex=True, inplace=True)
  df.replace(r'\t$', '', regex=True, inplace=True)
  #df.replace(r'^\'', '', regex=True, inplace=True)
  #df.replace(r'\'$', '', regex=True, inplace=True)
  return df

#TuringBench

In [ ]:
turingbench = load_dataset('turingbench/TuringBench',name='AA', split='train')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset turing_bench downloaded and prepared to /root/.cache/huggingface/datasets/turingbench___turing_bench/AA/1.1.0/d07d443078e98c47702ef8a6ab97bf1bd3c6d3a8664d463412a6ce774f670840. Subsequent calls will reuse this data.


In [ ]:
turingbench = pd.DataFrame.from_dict(turingbench)
turingbench

,Generation,label
0,' of paying more than ps1bn in fines to insure...,gpt2_pytorch
1,'dead sea shrinking by 1 meter every year the ...,gpt2_small
2,a blow to the head makes an instant hero in in...,fair_wmt19
3,"' ethereum release was released on april 18th,...",gpt2_pytorch
4,government sets up co-ordination panel to spee...,fair_wmt19
...,...,...
112199,'reliance fixed horizon fund ix xvi - ivd non-...,grover_large
112200,'spring pheasant count in north dakota provide...,xlnet_base
112201,'the city's civil rights division has agreed t...,gpt2_pytorch
112202,'hsbc banker arrested at jfk airport for money...,gpt2_large


In [ ]:
turingbench.label.value_counts()

gpt2_pytorch    5964
human           5964
pplm_gpt2       5964
grover_base     5964
xlm             5964
xlnet_base      5964
pplm_distil     5964
fair_wmt19      5507
gpt2_large      5507
gpt3            5507
gpt2_small      5507
gpt2_medium     5507
grover_large    5507
grover_mega     5507
gpt1            5402
gpt2_xl         5402
fair_wmt20      5402
transfo_xl      5401
xlnet_large     5152
ctrl            5148
Name: label, dtype: int64

In [ ]:
turingbench.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112204 entries, 0 to 112203
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Generation  112204 non-null  object
 1   label       112204 non-null  object
dtypes: object(2)
memory usage: 1.7+ MB


In [ ]:
clear_dataset(turingbench)

,Generation,label
0,' of paying more than ps1bn in fines to insure...,gpt2_pytorch
1,'dead sea shrinking by 1 meter every year the ...,gpt2_small
2,a blow to the head makes an instant hero in in...,fair_wmt19
3,"' ethereum release was released on april 18th,...",gpt2_pytorch
4,government sets up co-ordination panel to spee...,fair_wmt19
...,...,...
112199,'reliance fixed horizon fund ix xvi - ivd non-...,grover_large
112200,'spring pheasant count in north dakota provide...,xlnet_base
112201,'the city's civil rights division has agreed t...,gpt2_pytorch
112202,'hsbc banker arrested at jfk airport for money...,gpt2_large


In [ ]:
turingbench.rename(columns={'Generation' : 'text'}, inplace=True)

In [ ]:
turingbench['length'] = turingbench['text'].str.split().apply(len)
turingbench['source'] = 'TuringBench_train'
turingbench['language'] = 'en'

In [ ]:
turingbench = turingbench[turingbench.length > 5].reset_index(drop=True)

In [ ]:
turingbench

,text,label,length,source,language
0,' of paying more than ps1bn in fines to insure...,gpt2_pytorch,186,TuringBench_train,en
1,'dead sea shrinking by 1 meter every year the ...,gpt2_small,181,TuringBench_train,en
2,a blow to the head makes an instant hero in in...,fair_wmt19,221,TuringBench_train,en
3,"' ethereum release was released on april 18th,...",gpt2_pytorch,63,TuringBench_train,en
4,government sets up co-ordination panel to spee...,fair_wmt19,212,TuringBench_train,en
...,...,...,...,...,...
112199,'reliance fixed horizon fund ix xvi - ivd non-...,grover_large,13,TuringBench_train,en
112200,'spring pheasant count in north dakota provide...,xlnet_base,20,TuringBench_train,en
112201,'the city's civil rights division has agreed t...,gpt2_pytorch,185,TuringBench_train,en
112202,'hsbc banker arrested at jfk airport for money...,gpt2_large,108,TuringBench_train,en


In [ ]:
turingbench = turingbench.groupby(['label']).sample(5000, random_state = 0).sample(frac=1., random_state = 0).reset_index(drop=True)

In [ ]:
turingbench.label.value_counts()

ctrl            5000
grover_mega     5000
xlnet_base      5000
pplm_distil     5000
fair_wmt19      5000
xlm             5000
gpt1            5000
gpt2_pytorch    5000
pplm_gpt2       5000
human           5000
gpt2_small      5000
gpt3            5000
gpt2_medium     5000
gpt2_xl         5000
grover_large    5000
fair_wmt20      5000
transfo_xl      5000
gpt2_large      5000
grover_base     5000
xlnet_large     5000
Name: label, dtype: int64

In [ ]:
turingbench['domain'] = 'news'
turingbench['topic'] = 'politics'
turingbench = turingbench[column_order]

In [ ]:
turingbench.to_csv(datapath + 'dataset/TuringBench.csv', index=False)

In [ ]:
del turingbench

#Grover

In [ ]:
!rm generator*
!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=base~dataset=p0.90.jsonl
!head -12000 /content/generator=base~dataset=p0.90.jsonl > grover_base_0.90.jsonl
!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=base~dataset=p0.92.jsonl
!head -12000 /content/generator=base~dataset=p0.92.jsonl > grover_base_0.92.jsonl
!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=base~dataset=p0.94.jsonl
!head -12000 /content/generator=base~dataset=p0.94.jsonl > grover_base_0.94.jsonl
!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=base~dataset=p0.96.jsonl
!head -12000 /content/generator=base~dataset=p0.96.jsonl > grover_base_0.96.jsonl
!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=base~dataset=p0.98.jsonl
!head -12000 /content/generator=base~dataset=p0.98.jsonl > grover_base_0.98.jsonl
!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=base~dataset=p1.00.jsonl
!head -12000 /content/generator=base~dataset=p1.00.jsonl > grover_base_1.00.jsonl

!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=medium~dataset=p0.90.jsonl
!head -12000 /content/generator=medium~dataset=p0.90.jsonl > grover_medium_0.90.jsonl
!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=medium~dataset=p0.92.jsonl
!head -12000 /content/generator=medium~dataset=p0.92.jsonl > grover_medium_0.92.jsonl
!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=medium~dataset=p0.94.jsonl
!head -12000 /content/generator=medium~dataset=p0.94.jsonl > grover_medium_0.94.jsonl
!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=medium~dataset=p0.96.jsonl
!head -12000 /content/generator=medium~dataset=p0.96.jsonl > grover_medium_0.96.jsonl
!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=medium~dataset=p0.98.jsonl
!head -12000 /content/generator=medium~dataset=p0.98.jsonl > grover_medium_0.98.jsonl
!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=medium~dataset=p1.00.jsonl
!head -12000 /content/generator=medium~dataset=p1.00.jsonl > grover_medium_1.00.jsonl

!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=mega~dataset=p0.90.jsonl
!head -12000 /content/generator=mega~dataset=p0.90.jsonl > grover_mega_0.90.jsonl
!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=mega~dataset=p0.92.jsonl
!head -12000 /content/generator=mega~dataset=p0.92.jsonl > grover_mega_0.92.jsonl
!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=mega~dataset=p0.94.jsonl
!head -12000 /content/generator=mega~dataset=p0.94.jsonl > grover_mega_0.94.jsonl
!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=mega~dataset=p0.96.jsonl
!head -12000 /content/generator=mega~dataset=p0.96.jsonl > grover_mega_0.96.jsonl
!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=mega~dataset=p0.98.jsonl
!head -12000 /content/generator=mega~dataset=p0.98.jsonl > grover_mega_0.98.jsonl
!timeout 2s wget https://storage.googleapis.com/grover-models/generation_examples/generator=mega~dataset=p1.00.jsonl
!head -12000 /content/generator=mega~dataset=p1.00.jsonl > grover_mega_1.00.jsonl

rm: cannot remove 'generator*': No such file or directory
--2023-03-15 15:52:53--  https://storage.googleapis.com/grover-models/generation_examples/generator=base~dataset=p0.90.jsonl
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.135.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90266889 (86M) [application/octet-stream]
Saving to: ‘generator=base~dataset=p0.90.jsonl’

generator=base~data 100%[===================>]  86.08M   120MB/s    in 0.7s    

2023-03-15 15:52:54 (120 MB/s) - ‘generator=base~dataset=p0.90.jsonl’ saved [90266889/90266889]

--2023-03-15 15:52:54--  https://storage.googleapis.com/grover-models/generation_examples/generator=base~dataset=p0.92.jsonl
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.135.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.

In [ ]:
grover_files = glob.glob('grover_*.jsonl')
grover = pd.DataFrame()
for f in grover_files:
  temp = pd.read_json(f,lines=True)
  temp.loc[temp.label == 'machine', 'label'] = f.replace('.jsonl','')
  grover = pd.concat([grover, temp], ignore_index=True, copy=False)

In [ ]:
grover

,article,domain,title,date,authors,ind30k,url,label,orig_split,split,random_score,top_p
0,Patrons crowd the platform at the Washington M...,theepochtimes.com,US Senators Threaten Metro Funding Over Chines...,2019-04-15,None,29363,https://www.theepochtimes.com/us-senators-thre...,human,train_burner,train,-4.263557,NaN
1,Figure One: Oil Project Boom Levels\nThe chart...,seekingalpha.com,Supply Fundamentals Point To Continued Oil Rally,2019-04-12,Atlas Grinned,4999,https://seekingalpha.com/article/4254101-suppl...,grover_base_0.92,gen,train,NaN,0.920635
2,In December 2018 the issue of why the Canadian...,ottawacitizen.com,Military assessment ruling out the need for a ...,2019-04-16,Updated,23090,https://ottawacitizen.com/news/national/defenc...,human,train_burner,train,-3.863624,NaN
3,Modi also said the opposition wants to strip t...,deccanchronicle.com,Mahamilavat gang scared I will shut their shop...,2019-04-11,None,28881,https://www.deccanchronicle.com/nation/politic...,human,train_burner,train,-3.813725,NaN
4,"St Andrew's Special Emergency Response Unit, S...",jamaica-gleaner.com,Residents lament waterfront noise nuisance - K...,2019-04-10,Jason Cross Gleaner Writer,2274,http://beta.jamaica-gleaner.com/article/lead-s...,grover_base_0.92,gen,train,NaN,0.920635
...,...,...,...,...,...,...,...,...,...,...,...,...
215995,The April 2019 election victory is a remarkabl...,deccanchronicle.com,‘Magician’ Bibi likely to face stormy days ahead,2019-04-15,None,1918,https://www.deccanchronicle.com/opinion/column...,human,gen,val,0.470501,NaN
215996,"Kochi: Former and current politicians, bureauc...",deccanchronicle.com,‘Magician’ Bibi likely to face stormy days ahead,2019-04-15,None,1918,https://www.deccanchronicle.com/opinion/column...,grover_medium_0.96,gen,val,NaN,0.960317
215997,"DOVER, Del. — The remains of three Marines kil...",pennlive.com,Remains of 3 Marines killed in Afghanistan ret...,2019-04-12,The Associated Press,6612,https://www.pennlive.com/news/2019/04/remains-...,human,gen,val,0.470892,NaN
215998,WASHINGTON (AP) — More than a year after three...,pennlive.com,Remains of 3 Marines killed in Afghanistan ret...,2019-04-12,The Associated Press,6612,https://www.pennlive.com/news/2019/04/remains-...,grover_medium_0.96,gen,val,NaN,0.960317


In [ ]:
grover.label.value_counts()

human                 114192
grover_mega_0.98        5656
grover_medium_0.90      5656
grover_mega_0.90        5656
grover_base_0.98        5656
grover_base_0.94        5656
grover_mega_0.94        5656
grover_medium_1.00      5656
grover_base_0.90        5656
grover_base_1.00        5656
grover_base_0.92        5656
grover_medium_0.98      5656
grover_mega_0.92        5656
grover_mega_1.00        5656
grover_medium_0.92      5656
grover_base_0.96        5656
grover_medium_0.94      5656
grover_mega_0.96        5656
grover_medium_0.96      5656
Name: label, dtype: int64

In [ ]:
clear_dataset(grover)

,article,domain,title,date,authors,ind30k,url,label,orig_split,split,random_score,top_p
0,Patrons crowd the platform at the Washington M...,theepochtimes.com,US Senators Threaten Metro Funding Over Chines...,2019-04-15,None,29363,https://www.theepochtimes.com/us-senators-thre...,human,train_burner,train,-4.263557,NaN
1,Figure One: Oil Project Boom Levels\nThe chart...,seekingalpha.com,Supply Fundamentals Point To Continued Oil Rally,2019-04-12,Atlas Grinned,4999,https://seekingalpha.com/article/4254101-suppl...,grover_base_0.92,gen,train,NaN,0.920635
2,In December 2018 the issue of why the Canadian...,ottawacitizen.com,Military assessment ruling out the need for a ...,2019-04-16,Updated,23090,https://ottawacitizen.com/news/national/defenc...,human,train_burner,train,-3.863624,NaN
3,Modi also said the opposition wants to strip t...,deccanchronicle.com,Mahamilavat gang scared I will shut their shop...,2019-04-11,None,28881,https://www.deccanchronicle.com/nation/politic...,human,train_burner,train,-3.813725,NaN
4,"St Andrew's Special Emergency Response Unit, S...",jamaica-gleaner.com,Residents lament waterfront noise nuisance - K...,2019-04-10,Jason Cross Gleaner Writer,2274,http://beta.jamaica-gleaner.com/article/lead-s...,grover_base_0.92,gen,train,NaN,0.920635
...,...,...,...,...,...,...,...,...,...,...,...,...
215995,The April 2019 election victory is a remarkabl...,deccanchronicle.com,‘Magician’ Bibi likely to face stormy days ahead,2019-04-15,None,1918,https://www.deccanchronicle.com/opinion/column...,human,gen,val,0.470501,NaN
215996,"Kochi: Former and current politicians, bureauc...",deccanchronicle.com,‘Magician’ Bibi likely to face stormy days ahead,2019-04-15,None,1918,https://www.deccanchronicle.com/opinion/column...,grover_medium_0.96,gen,val,NaN,0.960317
215997,"DOVER, Del. — The remains of three Marines kil...",pennlive.com,Remains of 3 Marines killed in Afghanistan ret...,2019-04-12,The Associated Press,6612,https://www.pennlive.com/news/2019/04/remains-...,human,gen,val,0.470892,NaN
215998,WASHINGTON (AP) — More than a year after three...,pennlive.com,Remains of 3 Marines killed in Afghanistan ret...,2019-04-12,The Associated Press,6612,https://www.pennlive.com/news/2019/04/remains-...,grover_medium_0.96,gen,val,NaN,0.960317


In [ ]:
grover.rename(columns={'article' : 'text'}, inplace=True)

In [ ]:
grover['length'] = grover['text'].str.split().apply(len)
grover['source'] = 'grover_train'
grover['language'] = 'en'

In [ ]:
grover.drop(columns=['domain', 'title', 'date', 'authors', 'ind30k', 'url', 'orig_split', 'split', 'random_score', 'top_p'], inplace=True)

In [ ]:
grover = grover[grover.length > 5].reset_index(drop=True)

In [ ]:
grover

,text,label,length,source,language
0,Patrons crowd the platform at the Washington M...,human,184,grover_train,en
1,Figure One: Oil Project Boom Levels\nThe chart...,grover_base_0.92,672,grover_train,en
2,In December 2018 the issue of why the Canadian...,human,509,grover_train,en
3,Modi also said the opposition wants to strip t...,human,259,grover_train,en
4,"St Andrew's Special Emergency Response Unit, S...",grover_base_0.92,538,grover_train,en
...,...,...,...,...,...
215992,The April 2019 election victory is a remarkabl...,human,1032,grover_train,en
215993,"Kochi: Former and current politicians, bureauc...",grover_medium_0.96,287,grover_train,en
215994,"DOVER, Del. — The remains of three Marines kil...",human,463,grover_train,en
215995,WASHINGTON (AP) — More than a year after three...,grover_medium_0.96,383,grover_train,en


In [ ]:
grover.label.value_counts()

human                 114192
grover_base_0.92        5656
grover_medium_0.90      5656
grover_mega_0.90        5656
grover_base_0.94        5656
grover_medium_1.00      5656
grover_base_0.90        5656
grover_mega_0.98        5656
grover_base_1.00        5656
grover_medium_0.98      5656
grover_mega_0.92        5656
grover_mega_1.00        5656
grover_medium_0.92      5656
grover_base_0.96        5656
grover_medium_0.94      5656
grover_mega_0.96        5656
grover_medium_0.96      5656
grover_mega_0.94        5655
grover_base_0.98        5654
Name: label, dtype: int64

In [ ]:
grover = grover.groupby(['label']).sample(5000, random_state = 0).sample(frac=1., random_state = 0).reset_index(drop=True)

In [ ]:
grover.label.value_counts()

grover_base_0.90      5000
grover_base_0.96      5000
grover_base_0.98      5000
grover_medium_0.92    5000
human                 5000
grover_medium_1.00    5000
grover_medium_0.90    5000
grover_mega_0.98      5000
grover_mega_1.00      5000
grover_mega_0.96      5000
grover_mega_0.92      5000
grover_base_0.92      5000
grover_medium_0.98    5000
grover_medium_0.94    5000
grover_mega_0.90      5000
grover_base_1.00      5000
grover_base_0.94      5000
grover_mega_0.94      5000
grover_medium_0.96    5000
Name: label, dtype: int64

In [ ]:
grover['domain'] = 'news'
grover['topic'] = 'unknown'
grover = grover[column_order]

In [ ]:
grover.to_csv(datapath + 'dataset/Grover.csv', index=False)

In [ ]:
del grover

#GPT-2

In [ ]:
#https://github.com/openai/gpt-2-output-dataset/blob/master/detector/download.py
import os
import sys
import requests
from tqdm import tqdm

subdir = 'data'
if not os.path.exists(subdir):
    os.makedirs(subdir)
subdir = subdir.replace('\\','/') # needed for Windows

for ds in [
    'webtext',
    'small-117M',  'small-117M-k40',
    'medium-345M', 'medium-345M-k40',
    'large-762M',  'large-762M-k40',
    'xl-1542M',    'xl-1542M-k40',
]:
    for split in ['train']:
        filename = ds + "." + split + '.jsonl'
        r = requests.get("https://openaipublic.azureedge.net/gpt-2/output-dataset/v1/" + filename, stream=True)

        with open(os.path.join(subdir, filename), 'wb') as f:
            file_size = int(r.headers["content-length"])
            chunk_size = 1000
            with tqdm(ncols=100, desc="Fetching " + filename, total=file_size, unit_scale=True) as pbar:
                # 1k for chunk_size, since Ethernet packet size is around 1500 bytes
                for chunk in r.iter_content(chunk_size=chunk_size):
                    f.write(chunk)
                    pbar.update(chunk_size)


Fetching webtext.train.jsonl: 679Mit [00:49, 13.8Mit/s]                                             
Fetching small-117M.train.jsonl: 775Mit [00:36, 21.1Mit/s]                                          
Fetching small-117M-k40.train.jsonl: 711Mit [00:39, 17.9Mit/s]                                      
Fetching medium-345M.train.jsonl: 791Mit [00:37, 21.3Mit/s]                                         
Fetching medium-345M-k40.train.jsonl: 784Mit [01:53, 6.93Mit/s]                                     
Fetching large-762M.train.jsonl: 753Mit [00:43, 17.4Mit/s]                                          
Fetching large-762M-k40.train.jsonl: 751Mit [00:33, 22.2Mit/s]                                      
Fetching xl-1542M.train.jsonl: 724Mit [00:33, 21.9Mit/s]                                            
Fetching xl-1542M-k40.train.jsonl: 748Mit [00:23, 31.6Mit/s]                                        


In [ ]:
gpt2_files = glob.glob('data/*.jsonl')
gpt2_files.remove('data/webtext.train.jsonl')
gpt2 = pd.read_json('data/webtext.train.jsonl',lines=True)
gpt2['label'] = 'human'
gpt2 = gpt2.sample(10000, random_state=0)
for f in gpt2_files:
  temp = pd.read_json(f,lines=True)
  temp['label'] = 'gpt2-' + f.replace('data/','').replace('.train.jsonl','')
  temp.dropna(inplace=True)
  temp = temp.sample(10000, random_state=0)
  gpt2 = pd.concat([gpt2, temp], ignore_index=True, copy=False)

In [ ]:
gpt2['label'] = gpt2.label.str.replace('-','_')
gpt2.label.value_counts()

human                   10000
gpt2_xl_1542M           10000
gpt2_small_117M_k40     10000
gpt2_medium_345M_k40    10000
gpt2_xl_1542M_k40       10000
gpt2_large_762M         10000
gpt2_medium_345M        10000
gpt2_large_762M_k40     10000
gpt2_small_117M         10000
Name: label, dtype: int64

In [ ]:
clear_dataset(gpt2)

,id,ended,length,text,label
0,140089,True,164,Community Rating:\n\nCommunity Rating: 4.582 /...,human
1,93747,True,635,I own Tuscan Leather.\n\n\nI also own Pal Zile...,human
2,180647,True,76,"""This 'alt-right' brand is embracing extremism...",human
3,64526,True,748,Raising oil production during a supply glut an...,human
4,19100,False,1024,"�What are you doing?� �I�m walking,� a smooth ...",human
...,...,...,...,...,...
89995,71531,True,476,Bear In High School\n\nIncoming freshman Jorda...,gpt2_small_117M
89996,182747,True,2,benchmarks,gpt2_small_117M
89997,126148,False,1024,"In these classes, I demonstrate how laying sil...",gpt2_small_117M
89998,195623,True,59,It doesn't take long before I get a pair of ha...,gpt2_small_117M


In [ ]:
gpt2['length'] = gpt2['text'].str.split().apply(len)
gpt2['source'] = 'gpt2_train'
gpt2['language'] = 'en'
gpt2.drop(columns=['id', 'ended'], inplace=True)
gpt2 = gpt2[['text', 'label', 'length', 'source', 'language']]

In [ ]:
gpt2 = gpt2[gpt2.length > 5].reset_index(drop=True)

In [ ]:
gpt2

,text,label,length,source,language
0,Community Rating:\n\nCommunity Rating: 4.582 /...,human,128,gpt2_train,en
1,I own Tuscan Leather.\n\n\nI also own Pal Zile...,human,484,gpt2_train,en
2,"""This 'alt-right' brand is embracing extremism...",human,64,gpt2_train,en
3,Raising oil production during a supply glut an...,human,593,gpt2_train,en
4,"�What are you doing?� �I�m walking,� a smooth ...",human,772,gpt2_train,en
...,...,...,...,...,...
89754,"you don't know nothing once dose, sedation you...",gpt2_small_117M,807,gpt2_train,en
89755,Bear In High School\n\nIncoming freshman Jorda...,gpt2_small_117M,391,gpt2_train,en
89756,"In these classes, I demonstrate how laying sil...",gpt2_small_117M,852,gpt2_train,en
89757,It doesn't take long before I get a pair of ha...,gpt2_small_117M,50,gpt2_train,en


In [ ]:
gpt2 = gpt2.groupby(['label']).sample(5000, random_state = 0).sample(frac=1., random_state = 0).reset_index(drop=True)

In [ ]:
gpt2.label.value_counts()

human                   5000
gpt2_medium_345M        5000
gpt2_small_117M_k40     5000
gpt2_large_762M_k40     5000
gpt2_xl_1542M           5000
gpt2_medium_345M_k40    5000
gpt2_xl_1542M_k40       5000
gpt2_large_762M         5000
gpt2_small_117M         5000
Name: label, dtype: int64

In [ ]:
gpt2['domain'] = 'webtexts'
gpt2['topic'] = 'unknown'
gpt2 = gpt2[column_order]

In [ ]:
gpt2.to_csv(datapath + 'dataset/GPT-2.csv', index=False)

In [ ]:
del gpt2

#Paired Machine-Human
* CNN / Daily Mail + Grover Mega
* Writing Prompts + Fusion Model
* CnDARIO + GPT2

In [ ]:
import gdown
file_id="1xA9TtDYJE9BEwecL8QJ5d0LTytn5hhBr"
url = f'https://drive.google.com/uc?id={file_id}'
output = 'Paired_machine_and_human_generated_texts_with_likelihood_ranks.zip'
gdown.download(url, output, quiet=False)

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1xA9TtDYJE9BEwecL8QJ5d0LTytn5hhBr 



In [11]:
#if above not working, add shortcut to own gDrive directory and afterwards can copy
shutil.copy(datapath + 'dataset/Paired_machine_and_human_generated_texts_with_likelihood_ranks.zip', 'Paired_machine_and_human_generated_texts_with_likelihood_ranks.zip')
!unzip Paired_machine_and_human_generated_texts_with_likelihood_ranks.zip >/dev/null

'Paired_machine_and_human_generated_texts_with_likelihood_ranks.zip'

In [ ]:
with open('cn_novel_5billion/train.tgt') as f:
  text = f.readlines()
with open('cn_novel_5billion/train.label') as f:
  label = f.readlines()
cndario = pd.DataFrame({'text': text,'label': label}).sample(frac=1., random_state = 0).reset_index(drop=True)
cndario.loc[cndario.label.str.contains('1'), 'label'] = 'human'
cndario.loc[cndario.label.str.contains('0'), 'label'] = 'gpt2'

with open('en_grover/train.tgt') as f:
  text = f.readlines()
with open('en_grover/train.label') as f:
  label = f.readlines()
cnndaily = pd.DataFrame({'text': text,'label': label}).sample(frac=1., random_state = 0).reset_index(drop=True)
cnndaily.loc[cnndaily.label.str.contains('1'), 'label'] = 'human'
cnndaily.loc[cnndaily.label.str.contains('0'), 'label'] = 'grover'

with open('en_writing_prompt/train.tgt') as f:
  text = f.readlines()
with open('en_writing_prompt/train.label') as f:
  label = f.readlines()
wp = pd.DataFrame({'text': text,'label': label}).sample(frac=1., random_state = 0).reset_index(drop=True)
wp.loc[wp.label.str.contains('1'), 'label'] = 'human'
wp.loc[wp.label.str.contains('0'), 'label'] = 'fusion'

In [ ]:
clear_dataset(cndario)
cndario.drop_duplicates(inplace=True)
clear_dataset(cnndaily)
cnndaily.drop_duplicates(inplace=True)
clear_dataset(wp)
wp.drop_duplicates(inplace=True)

In [ ]:
cndario['length'] = cndario['text'].str.split().apply(len)
cndario['source'] = 'Paired_CnDARIO_train'
cndario['language'] = 'cn'

cnndaily['length'] = cnndaily['text'].str.split().apply(len)
cnndaily['source'] = 'Paired_CNN/DailyMail_train'
cnndaily['language'] = 'en'

wp['length'] = wp['text'].str.split().apply(len)
wp['source'] = 'Paired_WritingPrompts_train'
wp['language'] = 'en'

In [ ]:
cndario = cndario[cndario.length > 5].reset_index(drop=True)
cnndaily = cnndaily[cnndaily.length > 5].reset_index(drop=True)
wp = wp[wp.length > 5].reset_index(drop=True)

In [ ]:
cndario = cndario.groupby(['label']).sample(5000, random_state = 0).sample(frac=1., random_state = 0).reset_index(drop=True)
cnndaily = cnndaily.groupby(['label']).sample(5000, random_state = 0).sample(frac=1., random_state = 0).reset_index(drop=True)
wp = wp.groupby(['label']).sample(5000, random_state = 0).sample(frac=1., random_state = 0).reset_index(drop=True)

In [ ]:
print(cndario.label.value_counts())
print(cnndaily.label.value_counts())
print(wp.label.value_counts())

human    5000
gpt2     5000
Name: label, dtype: int64
human     5000
grover    5000
Name: label, dtype: int64
human     5000
fusion    5000
Name: label, dtype: int64


In [ ]:
cndario['domain'] = 'literature'
cndario['topic'] = 'unknown'
cndario = cndario[column_order]

cnndaily['domain'] = 'news'
cnndaily['topic'] = 'unknown'
cnndaily = cnndaily[column_order]

wp['domain'] = 'stories'
wp['topic'] = 'unknown'
wp = wp[column_order]

In [ ]:
cndario.to_csv(datapath + 'dataset/Paired_CnDARIO.csv', index=False)
cnndaily.to_csv(datapath + 'dataset/Paired_CNN_DailyMail.csv', index=False)
wp.to_csv(datapath + 'dataset/Paired_WritingPrompts.csv', index=False)

In [ ]:
del cndario
del cnndaily
del wp

#CNN / Daily Mail

In [ ]:
cnn = load_dataset('cnn_dailymail', '3.0.0', split='train[:10%]')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


In [ ]:
cnn = pd.DataFrame.from_dict(cnn)
cnn

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a
...,...,...,...
28706,"LOS ANGELES, California (CNN) -- The Beverly H...",Beverly Hills surgeon is in also in trouble ov...,50ef07a36a63a068324ce128ccf29fcacea8741b
28707,"(CNN) -- Authorities in Mobile, Alabama, are i...",The University of South Alabama says the stude...,9ed75b0b15c5f520b3664c84ef7aa37195352a95
28708,Jerusalem (CNN) -- The Israeli air force shot ...,"Israel views drone flyover ""very severely,"" De...",3bc80b7be9c60a45079101295374f2d11ab6f1da
28709,"LONDON, England (CNN) -- British authorities t...",Police examining video of Amy Winehouse smokin...,c4cdedd4e9539c303d0c677144f9da83bb7405a9


In [ ]:
clear_dataset(cnn)

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a
...,...,...,...
28706,"LOS ANGELES, California (CNN) -- The Beverly H...",Beverly Hills surgeon is in also in trouble ov...,50ef07a36a63a068324ce128ccf29fcacea8741b
28707,"(CNN) -- Authorities in Mobile, Alabama, are i...",The University of South Alabama says the stude...,9ed75b0b15c5f520b3664c84ef7aa37195352a95
28708,Jerusalem (CNN) -- The Israeli air force shot ...,"Israel views drone flyover ""very severely,"" De...",3bc80b7be9c60a45079101295374f2d11ab6f1da
28709,"LONDON, England (CNN) -- British authorities t...",Police examining video of Amy Winehouse smokin...,c4cdedd4e9539c303d0c677144f9da83bb7405a9


In [ ]:
cnn.drop(columns=['id', 'highlights'], inplace=True)

In [ ]:
cnn.rename(columns={'article' : 'text'}, inplace=True)

In [ ]:
cnn['label'] = 'human'

In [ ]:
cnn['length'] = cnn['text'].str.split().apply(len)
cnn['source'] = 'CNN_DailyMail_train'
cnn['language'] = 'en'

In [ ]:
cnn = cnn[['text', 'label', 'length', 'source', 'language']]

In [ ]:
cnn = cnn[cnn.length > 5].reset_index(drop=True)

In [ ]:
cnn

,text,label,length,source,language
0,"LONDON, England (Reuters) -- Harry Potter star...",human,455,CNN_DailyMail_train,en
1,Editor's note: In our Behind the Scenes series...,human,698,CNN_DailyMail_train,en
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...",human,743,CNN_DailyMail_train,en
3,WASHINGTON (CNN) -- Doctors removed five small...,human,414,CNN_DailyMail_train,en
4,(CNN) -- The National Football League has ind...,human,973,CNN_DailyMail_train,en
...,...,...,...,...,...
28706,"LOS ANGELES, California (CNN) -- The Beverly H...",human,495,CNN_DailyMail_train,en
28707,"(CNN) -- Authorities in Mobile, Alabama, are i...",human,299,CNN_DailyMail_train,en
28708,Jerusalem (CNN) -- The Israeli air force shot ...,human,194,CNN_DailyMail_train,en
28709,"LONDON, England (CNN) -- British authorities t...",human,392,CNN_DailyMail_train,en


In [ ]:
cnn = cnn.groupby(['label']).sample(5000, random_state = 0).sample(frac=1., random_state = 0).reset_index(drop=True)

In [ ]:
cnn.label.value_counts()

human    5000
Name: label, dtype: int64

In [ ]:
cnn['domain'] = 'news'
cnn['topic'] = 'unknown'
cnn = cnn[column_order]

In [ ]:
cnn.to_csv(datapath + 'dataset/CNN_DailyMail.csv', index=False)

In [ ]:
del cnn

#NeuralNews
* human from GoodNews
* machine from Grover

In [ ]:
import gdown
file_id="1yttGOSGvKQtPuLrJMJPByBlRfVQYisG1"
url = f'https://drive.google.com/uc?id={file_id}'
output = 'NeuralNews.zip'
gdown.download(url, output, quiet=False)

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1yttGOSGvKQtPuLrJMJPByBlRfVQYisG1 



In [ ]:
#if above not working, add shortcut to own gDrive directory and afterwards can copy
shutil.copy(datapath + 'dataset/NeuralNews.zip', 'NeuralNews.zip')
!unzip NeuralNews.zip >/dev/null

In [ ]:
files = glob.glob('NeuralNews/NeuralNews/fake_arts/*')
neuralnews = pd.DataFrame()
for file in files:
  with open(file) as f:
    text = f.read() #f.readlines()
  temp = pd.DataFrame({'text': text,'label': 'grover'}, index=[0])
  neuralnews = pd.concat([neuralnews, temp], ignore_index=True, copy=False)
neuralnews_machine = neuralnews

In [ ]:
files = glob.glob('NeuralNews/NeuralNews/real_arts/*')
neuralnews = pd.DataFrame()
for file in files:
  with open(file) as f:
    text = f.read() #f.readlines()
  temp = pd.DataFrame({'text': text,'label': 'human'}, index=[0])
  neuralnews = pd.concat([neuralnews, temp], ignore_index=True, copy=False)
neuralnews_human = neuralnews

In [ ]:
neuralnews = pd.concat([neuralnews_human, neuralnews_machine], ignore_index=True, copy=False)

In [ ]:
clear_dataset(neuralnews)

,text,label
0,"""The Spoils of Babylon"" seems as if it ought t...",human
1,"Trouble breaks in great, surging waves upon th...",human
2,Have you learned about climate change in your ...,human
3,"As a conceptual artist, Yoko Ono practices a v...",human
4,The shame is so blistering it drives Nathan to...,human
...,...,...
63995,A new report indicates that Dropbox has launch...,grover
63996,This year’s West Indian Day Parade in Brooklyn...,grover
63997,Sen. Marco Rubio (R-Fla.) has been entertainin...,grover
63998,To paraphrase the Bernie Sanders campaign slog...,grover


In [ ]:
neuralnews['length'] = neuralnews['text'].str.split().apply(len)
neuralnews['source'] = 'NeuralNews'
neuralnews['language'] = 'en'

In [ ]:
neuralnews = neuralnews[neuralnews.length > 5].reset_index(drop=True)

In [ ]:
neuralnews = neuralnews.groupby(['label']).sample(5000, random_state = 0).sample(frac=1., random_state = 0).reset_index(drop=True)

In [ ]:
neuralnews.label.value_counts()

human     5000
grover    5000
Name: label, dtype: int64

In [ ]:
neuralnews

,text,label,length,source,language
0,"""She was way too divisive and loud among other...",human,192,NeuralNews,en
1,"For nearly three seasons now, the Yankees have...",grover,240,NeuralNews,en
2,"As for the beard, “I heard about it from my mo...",grover,191,NeuralNews,en
3,PARIS -- At 9:30 on a nippy late-November morn...,human,275,NeuralNews,en
4,When the Toronto Maple Leafs ended their 78-ye...,grover,424,NeuralNews,en
...,...,...,...,...,...
9995,ATLANTA -- They are the accidental playoff par...,human,173,NeuralNews,en
9996,"On their new series, “Trollhunters,” creators ...",grover,376,NeuralNews,en
9997,Nothing happened against Ohio State on Thursda...,grover,388,NeuralNews,en
9998,TORONTO -- Women accounted for eight of the 10...,human,182,NeuralNews,en


In [ ]:
neuralnews['domain'] = 'news'
neuralnews['topic'] = 'unknown'
neuralnews = neuralnews[column_order]

In [ ]:
neuralnews.to_csv(datapath + 'dataset/NeuralNews.csv', index=False)

In [ ]:
del neuralnews

#In the Wild
* ArticleForge
* kafkai
* Reddit bot GPT3
* AI writer

In [ ]:
#dataset needed to be requested from authors
shutil.copy(datapath + 'dataset/InTheWild.zip', 'InTheWild.zip')
!unzip InTheWild.zip >/dev/null

In [ ]:
articleforge = pd.read_json('In_The_Wild/ArticleForge.jsonl', lines=True)
articleforge.loc[articleforge.label == 'machine', 'label'] = 'articleforge'
articleforge.rename(columns={'article' : 'text'}, inplace=True)
articleforge.drop(columns=['domain', 'title', 'date', 'authors', 'ind30k', 'url', 'orig_split', 'split'], inplace=True)
articleforge['domain'] = 'news'
articleforge['topic'] = 'unknown'

In [ ]:
aiwriter = pd.read_json('In_The_Wild/ai_writer.jsonl', lines=True)
aiwriter.loc[aiwriter.label == 'machine', 'label'] = 'ai_writer'
aiwriter.rename(columns={'article' : 'text'}, inplace=True)
aiwriter.drop(columns=['domain', 'title', 'data', 'authors', 'ind30k', 'url', 'orig_split', 'split', 'date'], inplace=True)
aiwriter['domain'] = 'news'
aiwriter['topic'] = 'unknown'

In [ ]:
kafkai = pd.read_json('In_The_Wild/kafkai.jsonl', lines=True)
kafkai.loc[kafkai.label == 'machine', 'label'] = 'kafkai'
kafkai.drop(columns=['id'], inplace=True)
kafkai['domain'] = 'articles'
kafkai['topic'] = 'unknown'

In [ ]:
redditbot = pd.read_json('In_The_Wild/reddit_bot_gpt3.jsonl', lines=True)
redditbot.loc[redditbot.label == 'machine', 'label'] = 'reddit_bot_gpt3'
redditbot.drop(columns=['id'], inplace=True)
redditbot['domain'] = 'comments'
redditbot['topic'] = 'unknown'

In [ ]:
inthewild = pd.concat([articleforge, aiwriter, kafkai, redditbot], ignore_index=True, copy=False)

In [ ]:
clear_dataset(inthewild)

,text,label,domain,topic
0,Most people will be familiar with the New York...,articleforge,news,unknown
1,nytimes.com today.\nThe story of the Times' mo...,human,news,unknown
2,The Definers' Social Media Strategy Group has ...,articleforge,news,unknown
3,Matt Rhoades in 2012.\nRhoades and a number of...,human,news,unknown
4,The New York Times has come out with their lis...,articleforge,news,unknown
...,...,...,...,...
7769,The game League of Legends has the most toxic ...,reddit_bot_gpt3,comments,unknown
7770,The answer to this question is very simple and...,reddit_bot_gpt3,comments,unknown
7771,"This is a very interesting question, and I hav...",reddit_bot_gpt3,comments,unknown
7772,There are many examples of this happening in h...,reddit_bot_gpt3,comments,unknown


In [ ]:
inthewild.label.value_counts()

human              3887
articleforge       1000
ai_writer          1000
kafkai             1000
reddit_bot_gpt3     887
Name: label, dtype: int64

In [ ]:
inthewild['length'] = inthewild['text'].str.split().apply(len)
inthewild['source'] = 'In the Wild'
inthewild['language'] = 'en'

In [ ]:
inthewild = inthewild[inthewild.length > 5].reset_index(drop=True)

In [ ]:
inthewild = inthewild.sample(frac=1., random_state = 0).reset_index(drop=True)

In [ ]:
inthewild

,text,label,domain,topic,length,source,language
0,My best friend committed suicide when we were ...,human,comments,unknown,172,In the Wild,en
1,"Set points, I know it's sometimes hard to find...",human,comments,unknown,191,In the Wild,en
2,I haven't been actively suicidal in three year...,human,comments,unknown,232,In the Wild,en
3,Published in the January/February 2015 issue B...,human,news,unknown,415,In the Wild,en
4,"So, I agree that there are many coincidences i...",reddit_bot_gpt3,comments,unknown,185,In the Wild,en
...,...,...,...,...,...,...,...
7769,Have you ever heard of the H2O Truth? It is a ...,kafkai,articles,unknown,398,In the Wild,en
7770,DIY investing platforms and tools will evolve ...,ai_writer,news,unknown,858,In the Wild,en
7771,The hotly anticipated remake of Annie is one o...,human,news,unknown,754,In the Wild,en
7772,Nicole Kidman has revealed she won't let her y...,human,news,unknown,675,In the Wild,en


In [ ]:
inthewild = inthewild[column_order]

In [ ]:
inthewild.to_csv(datapath + 'dataset/InTheWild.csv', index=False)

In [ ]:
del inthewild

#Wikitext

In [ ]:
wikitext = load_dataset('wikitext', 'wikitext-2-v1', split='train')

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


In [ ]:
wikitext = pd.DataFrame.from_dict(wikitext)
wikitext

,text
0,
1,= Valkyria Chronicles III = \n
2,
3,Senjō no Valkyria 3 : <unk> Chronicles ( Japa...
4,"The game began development in 2010 , carrying..."
...,...
36713,Common starlings may be kept as pets or as la...
36714,The common starling 's gift for mimicry has l...
36715,Mozart had a pet common starling which could ...
36716,Common starlings are trapped for food in some...


In [ ]:
clear_dataset(wikitext)

,text
0,
1,= Valkyria Chronicles III =\n
2,
3,Senjō no Valkyria 3 : <unk> Chronicles ( Japan...
4,"The game began development in 2010 , carrying ..."
...,...
36713,Common starlings may be kept as pets or as lab...
36714,The common starling 's gift for mimicry has lo...
36715,Mozart had a pet common starling which could s...
36716,Common starlings are trapped for food in some ...


In [ ]:
wikitext['label'] = 'human'

In [ ]:
wikitext['length'] = wikitext['text'].str.split().apply(len)
wikitext['source'] = 'Wikitext-2_train'
wikitext['language'] = 'en'

In [ ]:
wikitext.describe()

,length
count,36718.000000
mean,55.882946
std,77.557130
min,0.000000
25%,0.000000
50%,8.000000
75%,102.000000
max,699.000000


In [ ]:
wikitext.dropna(inplace=True)
wikitext = wikitext[wikitext.length > 5].reset_index(drop=True)

In [ ]:
wikitext

,text,label,length,source,language
0,Senjō no Valkyria 3 : <unk> Chronicles ( Japan...,human,127,Wikitext-2_train,en
1,"The game began development in 2010 , carrying ...",human,91,Wikitext-2_train,en
2,"It met with positive sales in Japan , and was ...",human,101,Wikitext-2_train,en
3,"As with previous <unk> Chronicles games , Valk...",human,211,Wikitext-2_train,en
4,"The game 's battle system , the <unk> system ,...",human,296,Wikitext-2_train,en
...,...,...,...,...,...
21219,= = = In science and culture = = =\n,human,10,Wikitext-2_train,en
21220,Common starlings may be kept as pets or as lab...,human,143,Wikitext-2_train,en
21221,The common starling 's gift for mimicry has lo...,human,149,Wikitext-2_train,en
21222,Mozart had a pet common starling which could s...,human,175,Wikitext-2_train,en


In [ ]:
wikitext = wikitext.groupby(['label']).sample(5000, random_state = 0).sample(frac=1., random_state = 0).reset_index(drop=True)

In [ ]:
wikitext.label.value_counts()

human    5000
Name: label, dtype: int64

In [ ]:
wikitext['domain'] = 'wikitexts'
wikitext['topic'] = 'unknown'
wikitext = wikitext[column_order]

In [ ]:
wikitext.to_csv(datapath + 'dataset/Wikitext.csv', index=False)

In [ ]:
del wikitext

#Writing Promts

In [ ]:
!wget https://dl.fbaipublicfiles.com/fairseq/data/writingPrompts.tar.gz -O writingPrompts.tar.gz >/dev/null 2>&1
!tar xvzf writingPrompts.tar.gz >/dev/null 2>&1

rm: cannot remove 'writingPrompts.tar.gz': No such file or directory


In [ ]:
with open('writingPrompts/train.wp_target') as f:
  text = f.readlines()
writingprompts = pd.DataFrame({'text': text,'label': 'human'}).sample(10000, random_state = 0).sample(frac=1., random_state = 0).reset_index(drop=True)

In [ ]:
writingprompts

,text,label
0,* '' A sword and a man have but two ends . `` ...,human
1,I was asleep when Monica switched the lights o...,human
2,"Assured , Alice continued to unravel the packa...",human
3,<newline> L-sometimes persistence is pointless...,human
4,***The uncovered ones must die . *** <newline>...,human
...,...,...
9995,The humans and aliens stared awkwardly at one ...,human
9996,When I signed up to join the military I though...,human
9997,"“ So , Mr Wentworth , can you tell us why you ...",human
9998,"C , <newline> <newline> I was mad at you , I h...",human


In [ ]:
clear_dataset(writingprompts)

,text,label
0,* '' A sword and a man have but two ends . `` ...,human
1,I was asleep when Monica switched the lights o...,human
2,"Assured , Alice continued to unravel the packa...",human
3,<newline> L-sometimes persistence is pointless...,human
4,***The uncovered ones must die . *** <newline>...,human
...,...,...
9995,The humans and aliens stared awkwardly at one ...,human
9996,When I signed up to join the military I though...,human
9997,"“ So , Mr Wentworth , can you tell us why you ...",human
9998,"C , <newline> <newline> I was mad at you , I h...",human


In [ ]:
writingprompts['length'] = writingprompts['text'].str.split().apply(len)
writingprompts['source'] = 'WritingPrompts_target_train'
writingprompts['language'] = 'en'

In [ ]:
writingprompts.dropna(inplace=True)
writingprompts = writingprompts[writingprompts.length > 5].reset_index(drop=True)

In [ ]:
writingprompts = writingprompts.groupby(['label']).sample(5000, random_state = 0).sample(frac=1., random_state = 0).reset_index(drop=True)

In [ ]:
writingprompts.label.value_counts()

human    5000
Name: label, dtype: int64

In [ ]:
writingprompts['domain'] = 'stories'
writingprompts['topic'] = 'unknown'
writingprompts = writingprompts[column_order]

In [ ]:
writingprompts.to_csv(datapath + 'dataset/WritingPrompts.csv', index=False)

In [ ]:
del writingprompts

#CoAuthor

In [ ]:
!wget https://cs.stanford.edu/~minalee/zip/chi2022-coauthor-v1.0.zip
!unzip -q chi2022-coauthor-v1.0.zip
!rm chi2022-coauthor-v1.0.zip

--2023-03-15 16:15:39--  https://cs.stanford.edu/~minalee/zip/chi2022-coauthor-v1.0.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49956179 (48M) [application/zip]
Saving to: ‘chi2022-coauthor-v1.0.zip’

chi2022-coauthor-v1 100%[===================>]  47.64M  11.0MB/s    in 4.3s    

2023-03-15 16:15:44 (11.0 MB/s) - ‘chi2022-coauthor-v1.0.zip’ saved [49956179/49956179]



In [ ]:
#https://coauthor.stanford.edu/
import os
import json


def find_writing_sessions(dataset_dir):
    paths = [
        os.path.join(dataset_dir, path)
        for path in os.listdir(dataset_dir) 
        if path.endswith('jsonl')
    ]
    return paths


def read_writing_session(path):
    events = []
    with open(path, 'r') as f:
        for event in f:
            events.append(json.loads(event))
    return events


dataset_dir = './coauthor-v1.0'
paths = find_writing_sessions(dataset_dir)

def apply_ops(doc, mask, ops, source):
    original_doc = doc
    original_mask = mask

    new_doc = ''
    new_mask = ''
    for i, op in enumerate(ops):

        # Handle retain operation
        if 'retain' in op:
            num_char = op['retain']

            retain_doc = original_doc[:num_char]
            retain_mask = original_mask[:num_char]

            original_doc = original_doc[num_char:]
            original_mask = original_mask[num_char:]

            new_doc = new_doc + retain_doc
            new_mask = new_mask + retain_mask

        # Handle insert operation
        elif 'insert' in op:
            insert_doc = op['insert']

            insert_mask = 'U' * len(insert_doc)  # User
            if source == 'api':
                insert_mask = 'A' * len(insert_doc)  # API

            if isinstance(insert_doc, dict):
                if 'image' in insert_doc:
                    print('Skipping invalid object insertion (image)')
                else:
                    print('Ignore invalid insertions:', op)
                    # Ignore other invalid insertions
                    # Debug if necessary
                    pass
            else:
                new_doc = new_doc + insert_doc
                new_mask = new_mask + insert_mask

        # Handle delete operation
        elif 'delete' in op:
            num_char = op['delete']

            if original_doc:
                original_doc = original_doc[num_char:]
                original_mask = original_mask[num_char:]
            else:
                new_doc = new_doc[:-num_char]
                new_mask = new_mask[:-num_char]

        else:
            # Ignore other operations
            # Debug if necessary
            print('Ignore other operations:', op)
            pass

    final_doc = new_doc + original_doc
    final_mask = new_mask + original_mask
    return final_doc, final_mask
    
def get_text_and_mask(events, event_id, remove_prompt=True):
    prompt = events[0]['currentDoc'].strip()

    text = prompt
    mask = 'P' * len(prompt)  # Prompt
    for event in events[:event_id]:
        if 'ops' not in event['textDelta']:
            continue
        ops = event['textDelta']['ops']
        source = event['eventSource']
        text, mask = apply_ops(text, mask, ops, source)

    if remove_prompt:
        if 'P' not in mask:
            print('=' * 80)
            print('Could not find the prompt in the final text')
            print('-' * 80)
            print('Prompt:', prompt)
            print('-' * 80)
            print('Final text:', text)
        else:
            end_index = mask.rindex('P')
            text = text[end_index + 1:]
            mask = mask[end_index + 1:]

    return text, mask

In [ ]:
coauthor = pd.DataFrame()
for file in range(len(paths)):
  try:
    events = read_writing_session(paths[file])
  except:
    continue
  text, mask = get_text_and_mask(events, len(events), remove_prompt=False)
  if ('A' in mask) and ('U' in mask):
    temp = pd.DataFrame({'text': text,'label': 'coauthor_gpt3'}, index=[0])
    coauthor = pd.concat([coauthor, temp], ignore_index=True, copy=False)

Skipping invalid object insertion (image)


In [ ]:
coauthor

,text,label
0,Should College Athletes Be Paid?\n\nDo you thi...,coauthor_gpt3
1,Are We Being Bad Citizens If We Don't Keep Up ...,coauthor_gpt3
2,Should Schools Provide Free Pads and Tampons?\...,coauthor_gpt3
3,How Worried Should We Be About Screen Time Dur...,coauthor_gpt3
4,Should College Athletes Be Paid?\n\nDo you thi...,coauthor_gpt3
...,...,...
1385,Is It Selfish to Pursue Risky Sports Like Extr...,coauthor_gpt3
1386,"Following World War III, all the nations of th...",coauthor_gpt3
1387,Is It Wrong to Focus on Animal Welfare When Hu...,coauthor_gpt3
1388,"A woman has been dating guy after guy, but it ...",coauthor_gpt3


In [ ]:
clear_dataset(coauthor)

,text,label
0,Should College Athletes Be Paid?\n\nDo you thi...,coauthor_gpt3
1,Are We Being Bad Citizens If We Don't Keep Up ...,coauthor_gpt3
2,Should Schools Provide Free Pads and Tampons?\...,coauthor_gpt3
3,How Worried Should We Be About Screen Time Dur...,coauthor_gpt3
4,Should College Athletes Be Paid?\n\nDo you thi...,coauthor_gpt3
...,...,...
1385,Is It Selfish to Pursue Risky Sports Like Extr...,coauthor_gpt3
1386,"Following World War III, all the nations of th...",coauthor_gpt3
1387,Is It Wrong to Focus on Animal Welfare When Hu...,coauthor_gpt3
1388,"A woman has been dating guy after guy, but it ...",coauthor_gpt3


In [ ]:
coauthor['length'] = coauthor['text'].str.split().apply(len)
coauthor['source'] = 'CoAuthor_final_text'
coauthor['language'] = 'en'

In [ ]:
coauthor.dropna(inplace=True)
coauthor = coauthor[coauthor.length > 5].reset_index(drop=True)

In [ ]:
coauthor = coauthor.groupby(['label']).sample(frac=1., random_state = 0).reset_index(drop=True)

In [ ]:
coauthor.label.value_counts()

coauthor_gpt3    1390
Name: label, dtype: int64

In [ ]:
coauthor['domain'] = 'stories'
coauthor['topic'] = 'unknown'
coauthor = coauthor[column_order]

In [ ]:
coauthor.to_csv(datapath + 'dataset/CoAuthor.csv', index=False)

In [ ]:
del coauthor

#TweepFake


In [ ]:
!wget https://raw.githubusercontent.com/tizfa/tweepfake_deepfake_text_detection/master/data/splits/train.csv -O TweepFake.csv >/dev/null

In [ ]:
!head TweepFake.csv

screen_name;text;account.type;class_type
imranyebot;YEA now that note GOOD;bot;others
zawvrk;Listen to This Charming Man by The Smiths  https://t.co/r12OIXkfUO;human;human
zawarbot;wish i can i would be seeing other hoes on the worst part;bot;others
ahadsheriffbot;The decade in the significantly easier schedule I Don't like h…;bot;others
kevinhookebot;"""Theim class=\""alignnone size-full wp-image-6017 \"" src=\""https://t.co/LiAsQsbs99\"" alt=\""\"" /&gt;\r\n&lt;p id=\""dyD\""&gt;&lt;img class=\""alignnone size-full wp-image-6047 \"" src=""";bot;rnn
dril;matrix 2: pedaphile killer;human;human
whalefakes;the world needs more whale stories. I would love to know what whalefacts are hiding in them.;bot;gpt2
imranyebot;the tweet of time I don’t mean It’s got us free lacroix;bot;others
ninjasexparty;"nobody:


In [ ]:
tweepfake = pd.read_csv('TweepFake.csv', sep=';')

In [ ]:
tweepfake

,screen_name,text,account.type,class_type
0,imranyebot,YEA now that note GOOD,bot,others
1,zawvrk,Listen to This Charming Man by The Smiths htt...,human,human
2,zawarbot,wish i can i would be seeing other hoes on the...,bot,others
3,ahadsheriffbot,The decade in the significantly easier schedul...,bot,others
4,kevinhookebot,"""Theim class=\""alignnone size-full wp-image-60...",bot,rnn
...,...,...,...,...
20707,AINarendraModi,Met on the Abversion of our science for the co...,bot,rnn
20708,AINarendraModi,Land for their during the opportunity to the p...,bot,rnn
20709,DeepDrumpf,@TayandYou doesn't have a clue. You're right. ...,bot,rnn
20710,jaden,Me And My Bestie https://t.co/vPq2iDkWZm,human,human


In [ ]:
tweepfake['account.type'].value_counts()

human    10358
bot      10354
Name: account.type, dtype: int64

In [ ]:
tweepfake['class_type'].value_counts()

human     10358
others     3920
rnn        3325
gpt2       3109
Name: class_type, dtype: int64

In [ ]:
tweepfake['class_type'] = tweepfake['class_type'].replace('others','machine-others')

In [ ]:
tweepfake.rename(columns={'class_type' : 'label'}, inplace=True)

In [ ]:
tweepfake.drop(columns=['screen_name', 'account.type'], inplace=True)

In [ ]:
clear_dataset(tweepfake)

,text,label
0,YEA now that note GOOD,machine-others
1,Listen to This Charming Man by The Smiths htt...,human
2,wish i can i would be seeing other hoes on the...,machine-others
3,The decade in the significantly easier schedul...,machine-others
4,"""Theim class=\""alignnone size-full wp-image-60...",rnn
...,...,...
20707,Met on the Abversion of our science for the co...,rnn
20708,Land for their during the opportunity to the p...,rnn
20709,@TayandYou doesn't have a clue. You're right. ...,rnn
20710,Me And My Bestie https://t.co/vPq2iDkWZm,human


In [ ]:
tweepfake['length'] = tweepfake['text'].str.split().apply(len)
tweepfake['source'] = 'TweepFake_train'
tweepfake['language'] = 'en'

In [ ]:
tweepfake = tweepfake[tweepfake.length > 5].reset_index(drop=True)

In [ ]:
tweepfake.label.value_counts()

human             8554
rnn               3224
gpt2              2915
machine-others    2762
Name: label, dtype: int64

In [ ]:
tweepfake = tweepfake.groupby(['label']).sample(2750, random_state = 0).sample(frac=1., random_state = 0).reset_index(drop=True)

In [ ]:
tweepfake['domain'] = 'tweets'
tweepfake['topic'] = 'unknown'
tweepfake = tweepfake[column_order]

In [ ]:
tweepfake.to_csv(datapath + 'dataset/TweepFake.csv', index=False)

In [ ]:
del tweepfake

#WELFake
* real & fake (not necessarily machine-generated) news

In [ ]:
!wget https://zenodo.org/record/4561253/files/WELFake_Dataset.csv? -O WELFake_Dataset.csv

--2023-03-15 16:17:39--  https://zenodo.org/record/4561253/files/WELFake_Dataset.csv?
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 245086152 (234M) [text/plain]
Saving to: ‘WELFake_Dataset.csv’

WELFake_Dataset.csv 100%[===================>] 233.73M   424KB/s    in 10m 0s  

2023-03-15 16:27:41 (399 KB/s) - ‘WELFake_Dataset.csv’ saved [245086152/245086152]



In [ ]:
welfake = pd.read_csv('WELFake_Dataset.csv')

In [ ]:
welfake.drop(columns=['Unnamed: 0', 'title'], inplace=True)

In [ ]:
welfake.loc[welfake.label == 1, 'label'] = 'human_real'
welfake.loc[welfake.label == 0, 'label'] = 'human_fake'

In [ ]:
clear_dataset(welfake)

,text,label
0,No comment is expected from Barack Obama Membe...,human_real
1,Did they post their votes for Hillary already?,human_real
2,"Now, most of the demonstrators gathered last n...",human_real
3,A dozen politically active pastors came here f...,human_fake
4,"The RS-28 Sarmat missile, dubbed Satan 2, will...",human_real
...,...,...
72129,WASHINGTON (Reuters) - Hackers believed to be ...,human_fake
72130,"You know, because in fantasyland Republicans n...",human_real
72131,Migrants Refuse To Leave Train At Refugee Camp...,human_fake
72132,MEXICO CITY (Reuters) - Donald Trump’s combati...,human_fake


In [ ]:
welfake.dropna(inplace=True)

In [ ]:
welfake.text = welfake.text.astype('string')

In [ ]:
welfake['length'] = welfake['text'].str.split().apply(len)
welfake['source'] = 'WELFake'
welfake['language'] = 'en'

In [ ]:
welfake = welfake[welfake.length > 5].reset_index(drop=True)

In [ ]:
welfake.label.value_counts()

human_real    36079
human_fake    35025
Name: label, dtype: int64

In [ ]:
welfake = welfake.groupby(['label']).sample(5000, random_state = 0).sample(frac=1., random_state = 0).reset_index(drop=True)

In [ ]:
welfake['domain'] = 'news'
welfake['topic'] = 'unknown'
welfake = welfake[column_order]

In [ ]:
welfake.to_csv(datapath + 'dataset/WELFake.csv', index=False)

In [ ]:
del welfake

#WMT-19 ToDo (not enough space to download whole dataset using huggingface api)

In [ ]:
def load_language(language, cache_dir):
    # load either the english or german portion of the wmt16 dataset
    assert language in ['en', 'de']
    d = load_dataset('wmt19', 'de-en', split='train')
    docs = d['translation'][0:100000]
    desired_language_docs = [d[language] for d in docs]
    lens = [len(d.split()) for d in desired_language_docs]
    sub = [d for d, l in zip(desired_language_docs, lens) if l > 5 and l < 1000]
    return sub

In [ ]:
#sub = load_language('de','.')

In [ ]:
#import torch

# English to German translation
#en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de', checkpoint_file='model1.pt:model2.pt:model3.pt:model4.pt',
#                       tokenizer='moses', bpe='fastbpe')
#en2de.translate("Machine learning is great!")

# AuTexTification
*   English & Spanish



In [12]:
#requires registration at https://zenodo.org/record/7692961 to download dataset

In [13]:
att_en_train = pd.read_csv(datapath + 'AuTexTification/subtask_1/en/train.tsv.gz', sep='\t')
att_es_train = pd.read_csv(datapath + 'AuTexTification/subtask_1/es/train.tsv.gz', sep='\t')
att_en_train2 = pd.read_csv(datapath + 'AuTexTification/subtask_2/en/train.tsv.gz', sep='\t')
att_es_train2 = pd.read_csv(datapath + 'AuTexTification/subtask_2/es/train.tsv.gz', sep='\t')

In [14]:
att_en_train

,id,text,label
0,12322,you need to stop the engine and wait until it ...,generated
1,1682,The Commission shall publish the report; an in...,generated
2,22592,"I have not been tweeting a lot lately, but I d...",generated
3,17390,I pass my exam and really thankgod for that bu...,human
4,30453,The template will have 3 parts: a mustache sha...,human
...,...,...,...
33840,16850,"@PierreJoye i have a server already, thanks fo...",human
33841,6265,10. Article 13 of Council Directive 80/777/EEC...,human
33842,11284,"Crying because I have to cry for you?. No. No,...",generated
33843,860,"However, it will continue to offer customers a...",generated


In [ ]:
att_es_train

,id,text,label
0,5464,Entrada en vigor. La presente Directiva entrar...,human
1,30129,Preguntas: 1. ¿Cuáles son los principales argu...,generated
2,19553,¿Desea algo? Póngame una caja de madera. ¿Qué ...,generated
3,13005,"@victor28088 1665 Tweets no originales, que as...",human
4,16919,De pequeño Dios me dio a elegir entre tener un...,human
...,...,...,...
32057,16850,"Mamá, ¿por qué no me despertaste? Te hable 5 v...",human
32058,6265,. Artículo 2. Los Estados miembros aplicarán l...,human
32059,11284,Mi memoria es:  5% de los médicos tienen una ...,generated
32060,860,APROBAR el proyecto de resolución que se adjun...,generated


In [ ]:
att_en_train2

,id,text,label
0,6239,It was not until many years later that it coul...,A
1,9255,Users can then pin these images to their profi...,F
2,1674,The best songs are those that I can sing along...,B
3,5001,I found this book to be poorly written. It was...,D
4,20779,Regulates the application of the EU tariff quo...,E
...,...,...,...
22411,6915,The ministry had earlier said it was working w...,B
22412,14431,"Once combined, slowly add the melted butter, s...",F
22413,1288,Hightower for this new and wellwritten book!. ...,B
22414,5563,All throughout the whole book this author has ...,D


In [ ]:
att_es_train2

,id,text,label
0,12786,"Sin embargo, los jóvenes son capaces de recono...",B
1,12361,¿Hay algo más que quieras compartir? ¿Algo sob...,B
2,1662,"El servicio de sala es bueno, rápido y amabilí...",B
3,14729,"Para concentrarse en el hablante, trata de des...",F
4,9312,Los responsables locales tendrán ahora que esp...,F
...,...,...,...
21930,11706,Escribe un correo electrónico a PI:EMAIL y cué...,A
21931,7997,"Los osos negros y grizzlies de Yellowstone, ta...",C
21932,11369,"El lenguaje es un recurso para comunicarse, pe...",A
21933,25259,"Dr, creo que las ostras son animales No, las o...",F


In [ ]:
att_en_train['length'] = att_en_train['text'].str.split().apply(len)
att_en_train['source'] = 'AuTexTification'
att_en_train['language'] = 'en'

att_en_train2['length'] = att_en_train2['text'].str.split().apply(len)
att_en_train2['source'] = 'AuTexTification'
att_en_train2['language'] = 'en'

att_es_train['length'] = att_es_train['text'].str.split().apply(len)
att_es_train['source'] = 'AuTexTification'
att_es_train['language'] = 'es'

att_es_train2['length'] = att_es_train2['text'].str.split().apply(len)
att_es_train2['source'] = 'AuTexTification'
att_es_train2['language'] = 'es'

In [ ]:
att = pd.concat([att_en_train, att_en_train2, att_es_train, att_es_train2], ignore_index=True, copy=False)
att.drop(columns=['id'], inplace=True)
att = att[~att.duplicated(subset=['text'], keep='last')]

In [ ]:
att.label.value_counts()

human        32831
generated    17974
F             7693
D             7658
E             7590
C             7262
B             7162
A             6984
Name: label, dtype: int64

In [ ]:
att = clear_dataset(att)

In [ ]:
att = att[att.length > 5].reset_index(drop=True)

In [ ]:
att

,text,label,length,source,language
0,you need to stop the engine and wait until it ...,generated,23,AuTexTification,en
1,I pass my exam and really thankgod for that bu...,human,20,AuTexTification,en
2,The template will have 3 parts: a mustache sha...,human,59,AuTexTification,en
3,@Jonasbrothers Guys.. U rock!! I love all of y...,generated,21,AuTexTification,en
4,"2., paragraph 3., amend as follows: 3. In the ...",generated,81,AuTexTification,en
...,...,...,...,...,...
94985,Escribe un correo electrónico a PI:EMAIL y cué...,A,77,AuTexTification,es
94986,"Los osos negros y grizzlies de Yellowstone, ta...",C,78,AuTexTification,es
94987,"El lenguaje es un recurso para comunicarse, pe...",A,85,AuTexTification,es
94988,"Dr, creo que las ostras son animales No, las o...",F,22,AuTexTification,es


In [ ]:
att.loc[att.label.str.contains('generated'), 'label'] = 'machine'
att.loc[att.label.str.contains('A'), 'label'] = 'machine_A'
att.loc[att.label.str.contains('B'), 'label'] = 'machine_B'
att.loc[att.label.str.contains('C'), 'label'] = 'machine_C'
att.loc[att.label.str.contains('D'), 'label'] = 'machine_D'
att.loc[att.label.str.contains('E'), 'label'] = 'machine_E'
att.loc[att.label.str.contains('F'), 'label'] = 'machine_F'

In [ ]:
att = att.groupby(['label']).sample(5000, random_state = 0).sample(frac=1., random_state = 0).reset_index(drop=True)

In [ ]:
att['domain'] = 'articles_social_media'
att['topic'] = 'legal_how-to'
att = att[column_order]

In [ ]:
att.to_csv(datapath + 'dataset/AuTexTification.csv', index=False)

In [ ]:
del att
del att_en_train
del att_en_train2
del att_es_train
del att_es_train2

# MTG
*   tasks: story generation, question generation, title generation and text summarization
*   languages: English, German, French, Spanish and Chinese



In [ ]:
#add shortcut to https://drive.google.com/file/d/1bmfn5nhGeXktR4WuK6rzckexGl7gQo3S/view to own gDrive directory and afterwards can copy
shutil.copy(datapath + 'dataset/MTG/ALL.zip', 'ALL.zip')
!unzip ALL.zip > /dev/null

In [ ]:
def load_MTG_language(language):
  label = 'human' if language == 'en'else 'corrected_google_translate'
  df = pd.DataFrame()
  #QG
  with open('ALL/QG/train.annotation.question.' + language) as f:
    text = f.readlines() 
  df = pd.concat([df, pd.DataFrame({'text': text, 'source': 'MGT_QG_train.annotation.question', 'domain': 'question_answering', 'topic': 'wikitexts'})])
  with open('ALL/QG/train.annotation.passage.' + language) as f:
    text = f.readlines() 
  df = pd.concat([df, pd.DataFrame({'text': text, 'source': 'MGT_QG_train.annotation.passage', 'domain': 'question_answering', 'topic': 'wikitexts'})])
  with open('ALL/QG/train.annotation.answer.' + language) as f:
    text = f.readlines() 
  df = pd.concat([df, pd.DataFrame({'text': text, 'source': 'MGT_QG_train.annotation.answer', 'domain': 'question_answering', 'topic': 'wikitexts'})])
  #SG
  with open('ALL/SG/train.annotation.story.src.' + language) as f:
    text = f.readlines() 
  df = pd.concat([df, pd.DataFrame({'text': text, 'source': 'MGT_SG_train.annotation.story.src', 'domain': 'stories', 'topic': 'daily_life'})])
  with open('ALL/SG/train.annotation.story.trg.' + language) as f:
    text = f.readlines() 
  df = pd.concat([df, pd.DataFrame({'text': text, 'source': 'MGT_SG_train.annotation.story.trg', 'domain': 'stories', 'topic': 'daily_life'})])
  #Summ
  with open('ALL/Summ/train.annotation.doc.' + language) as f:
    text = f.readlines() 
  df = pd.concat([df, pd.DataFrame({'text': text, 'source': 'MGT_Summ_train.annotation.doc', 'domain': 'news', 'topic': 'unknown'})])
  with open('ALL/Summ/train.annotation.sum.' + language) as f:
    text = f.readlines() 
  df = pd.concat([df, pd.DataFrame({'text': text, 'source': 'MGT_Summ_train.annotation.sum', 'domain': 'news', 'topic': 'unknown'})])
  #TG
  with open('ALL/TG/train.annotation.doc.' + language) as f:
    text = f.readlines() 
  df = pd.concat([df, pd.DataFrame({'text': text, 'source': 'MGT_TG_train.annotation.doc', 'domain': 'news', 'topic': 'unknown'})])
  with open('ALL/TG/train.annotation.title.' + language) as f:
    text = f.readlines() 
  df = pd.concat([df, pd.DataFrame({'text': text, 'source': 'MGT_TG_train.annotation.title', 'domain': 'news', 'topic': 'unknown'})])
  
  df.replace(r'\n$', '', regex=True, inplace=True)
  clear_dataset(df)
  df['label'] = label
  df['length'] = df['text'].str.split().apply(len)
  df['language'] = language
  df = df[column_order]
  df = df[df.length > 5].reset_index(drop=True)
  df = df.groupby(['source']).apply(lambda x: x.sample(min(5000, len(x)), random_state = 0)).sample(frac=1., random_state = 0).reset_index(drop=True)
  return df

In [ ]:
mtg_en = load_MTG_language('en')
mtg_de = load_MTG_language('de')
mtg_fr = load_MTG_language('fr')
mtg_es = load_MTG_language('es')
mtg_zh = load_MTG_language('zh') #chinese

In [ ]:
mtg_zh

,text,label,length,source,language,domain,topic
0,今天早上，宠物宠物的股票上涨，零售商让一些投资者放在所有部分的增长，零售商集团收入增长为12...,corrected_google_translate,6,MGT_TG_train.annotation.doc,zh,news,unknown
1,（cnn）本周在底特律以外的一个小社区中提出了一场戏剧性的事件，在底特律之外的一个小社区转向...,corrected_google_translate,13,MGT_Summ_train.annotation.doc,zh,news,unknown
2,在美洲的其他地方，特别是在加勒比地区，施洗者传教士在反奴隶制运动中发挥了积极作用。例如，在牙...,corrected_google_translate,7,MGT_QG_train.annotation.passage,zh,question_answering,wikitexts
3,英格兰中心manu tuilagi正在将他的康复'一周带到一周中，因为州长中场继续从一个麻烦...,corrected_google_translate,24,MGT_Summ_train.annotation.doc,zh,news,unknown
4,在与她的尸体发生性关系之前，这位20岁的疯狂披萨送货司机正在刺伤和扼杀他的室友死亡之前，被判...,corrected_google_translate,12,MGT_Summ_train.annotation.doc,zh,news,unknown
...,...,...,...,...,...,...,...
18129,哈桑·芒什（hassan munshi）是两名害怕进入叙利亚加入伊西斯的少年之一，据信是哈马...,corrected_google_translate,8,MGT_Summ_train.annotation.doc,zh,news,unknown
18130,twitter是唐纳德特朗普将媒体描绘为“假新闻”的画布，将俄罗斯传奇作为“女巫狩猎”和他的...,corrected_google_translate,12,MGT_TG_train.annotation.doc,zh,news,unknown
18131,一个42岁的男子瘫痪在犹他州邻里，观看射击，发现与一位17岁的女孩反复发生性关系是要求上诉法...,corrected_google_translate,10,MGT_Summ_train.annotation.doc,zh,news,unknown
18132,下午5点左右，一名无姓名的男子被发现死于明显的自己造成的伤口。 在星期天 。 事件发生在内华...,corrected_google_translate,8,MGT_Summ_train.annotation.sum,zh,news,unknown


In [ ]:
mtg = pd.concat([mtg_en, mtg_de, mtg_fr, mtg_es, mtg_zh], ignore_index=True, copy=False)

In [ ]:
mtg.label.value_counts()

corrected_google_translate    145456
human                          42157
Name: label, dtype: int64

In [ ]:
mtg.source.value_counts()

MGT_Summ_train.annotation.doc        25000
MGT_TG_train.annotation.doc          25000
MGT_QG_train.annotation.passage      24924
MGT_Summ_train.annotation.sum        23004
MGT_SG_train.annotation.story.src    20117
MGT_TG_train.annotation.title        20032
MGT_QG_train.annotation.question     20030
MGT_SG_train.annotation.story.trg    20004
MGT_QG_train.annotation.answer        9502
Name: source, dtype: int64

In [ ]:
mtg.language.value_counts()

es    42887
fr    42602
en    42157
de    41833
zh    18134
Name: language, dtype: int64

In [ ]:
mtg = mtg.groupby(['language']).sample(10000, random_state = 0).sample(frac=1., random_state = 0).reset_index(drop=True)

In [ ]:
mtg.to_csv(datapath + 'dataset/MTG.csv', index=False)

In [ ]:
del mtg
del mtg_en, mtg_de, mtg_fr, mtg_es, mtg_zh

#Combine Dataset

In [ ]:
def binarize_labels(df):
  df.loc[df.label.str.contains('human'), 'label'] = int(1)
  df.loc[df.label != 1, 'label'] = int(0)

In [ ]:
turingbench = pd.read_csv(datapath + 'dataset/TuringBench.csv')
grover = pd.read_csv(datapath + 'dataset/Grover.csv')
gpt2 = pd.read_csv(datapath + 'dataset/GPT-2.csv')
paired_cndario = pd.read_csv(datapath + 'dataset/Paired_CnDARIO.csv')
paired_cnndaily = pd.read_csv(datapath + 'dataset/Paired_CNN_DailyMail.csv')
paired_wp = pd.read_csv(datapath + 'dataset/Paired_WritingPrompts.csv')
cnn = pd.read_csv(datapath + 'dataset/CNN_DailyMail.csv')
neuralnews = pd.read_csv(datapath + 'dataset/NeuralNews.csv')
inthewild = pd.read_csv(datapath + 'dataset/InTheWild.csv')
wikitext = pd.read_csv(datapath + 'dataset/Wikitext.csv')
writingprompts = pd.read_csv(datapath + 'dataset/WritingPrompts.csv')
coauthor = pd.read_csv(datapath + 'dataset/CoAuthor.csv')
tweepfake = pd.read_csv(datapath + 'dataset/TweepFake.csv')
welfake = pd.read_csv(datapath + 'dataset/WELFake.csv')

att = pd.read_csv(datapath + 'dataset/AuTexTification.csv')
mtg = pd.read_csv(datapath + 'dataset/MTG.csv')

In [ ]:
all = [turingbench, grover, gpt2, paired_cndario, paired_cnndaily, paired_wp, cnn, neuralnews, inthewild, wikitext, writingprompts, coauthor, tweepfake, welfake]
for i in all:
  binarize_labels(i)

In [ ]:
train = pd.concat([grover, gpt2, cnn, wikitext, writingprompts, paired_cndario, paired_cnndaily, paired_wp], ignore_index=True, copy=False).groupby(['label']).sample(40000, random_state = 0).sample(frac=1., random_state = 0).reset_index(drop=True)
test = pd.concat([turingbench, neuralnews, inthewild, coauthor, tweepfake, welfake], ignore_index=True, copy=False).sample(frac=1., random_state = 0).reset_index(drop=True)

In [ ]:
train.label.value_counts()

0    40000
1    40000
Name: label, dtype: int64

In [ ]:
test.label.value_counts()

0    113527
1     26637
Name: label, dtype: int64

In [ ]:
train.to_csv(datapath + 'dataset/train.csv', index=False)
test.to_csv(datapath + 'dataset/test.csv', index=False)